In [25]:
import re
import math
import copy
from collections import deque

# Utils

In [26]:
def find_qubit(qubit, grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == qubit:
                return (i, j)
    return None

def ManDist(neighbor, qubit):
    return abs(neighbor[0] - qubit[0]) + abs(neighbor[1] - qubit[1]) - 1

def DoA(new_qubit, qubit_pos, grid, adj_mat):
    qubit = grid[qubit_pos[0]][qubit_pos[1]]
    return adj_mat[new_qubit-1][qubit-1] + adj_mat[qubit-1][new_qubit-1]

def update_neighbor_list(grid, X, Y, neighborList = set()):
    # Remove the occupied NN positions from neighborList.
    if (X, Y) in neighborList:
        neighborList.remove((X, Y))
    if Y > 0 and grid[X][Y-1] == 0:
        neighborList.add((X, Y-1))
    if X > 0 and grid[X-1][Y] == 0:
        neighborList.add((X-1, Y))
    if Y < len(grid[0])-1 and grid[X][Y+1] == 0:
        neighborList.add((X, Y+1))
    if X < len(grid)-1 and grid[X+1][Y] == 0:
        neighborList.add((X+1, Y))
    return neighborList

# Place Qubits

In [27]:
def place_qubits(pq, M, N, adj_mat):
    grid = [[0 for _ in range(N)] for _ in range(M)]
    placedQubitList = []

    # Place the first qubit in the center of the grid.
    grid[M//2][N//2] = pq.pop(0)
    placedQubitList.append((M//2, N//2))
    neighborList = update_neighbor_list(grid, M//2, N//2)
    # print(neighborList, placedQubitList)

    # Place the remaining qubits.
    while pq:
        q = pq.pop(0)
        minCost, minX, minY = int(1e9), int(1e9), int(1e9)
        for neighbor in neighborList:
            cost = 0
            for qubit in placedQubitList:
                cost += ManDist(neighbor, qubit) * DoA(q, qubit, grid, adj_mat)
            if cost < minCost:
                minCost = cost
                minX, minY = neighbor
            elif cost == minCost:
                if abs(neighbor[0] - M//2) + abs(neighbor[1] - N//2) < abs(minX - M//2) + abs(minY - N//2):
                    minX, minY = neighbor
            # print(f"qubit: {q}, neighbor: {neighbor}, cost: {cost}, minCost: {minCost}, minX: {minX}, minY: {minY}")
        grid[minX][minY] = q
        placedQubitList.append((minX, minY))
        neighborList = update_neighbor_list(grid, minX, minY, neighborList)
        # print(neighborList, placedQubitList)

    return grid


# Finding the best paths

In [28]:
# Check if cell(x,y) is in the grid
def is_valid_cell(grid, x, y):
    M, N = len(grid), len(grid[0])
    return 0 <= x < M and 0 <= y < N

def find_min_paths(grid, src, dest):
    M, N = len(grid), len(grid[0])
    directions = [(0, -1), (-1, 0), (0, 1), (1, 0)]
    queue = deque([(src, [src])])
    min_paths = []
    visited = set()

    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == dest:
            min_paths.append(path)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_cell(grid, nx, ny) and (nx, ny) not in visited:
                queue.append(((nx, ny), path + [(nx, ny)]))
        visited.add((x, y))
    return min_paths
    

# Rearrange Utils

In [29]:
def nPaths(p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    right_moves = abs(y1-y2)
    down_moves = abs(x1-x2)
    total_moves = right_moves + down_moves
    return int(math.factorial(total_moves) / (math.factorial(right_moves) * math.factorial(down_moves)))

def nTrace(p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    return abs(x1-x2) + abs(y1-y2) -1

def swap_qubits(grid, path):
    n = len(path) - 1
    grids = []
    for i in range(n):
        new_grid = copy.deepcopy(grid)
        swap_src, swap_dest = i, n-i-1
        count = 0

        for j in range(swap_src):
            x1, y1 = path[j][0], path[j][1]
            x2, y2 = path[j+1][0], path[j+1][1]
            new_grid[x1][y1], new_grid[x2][y2] = new_grid[x2][y2], new_grid[x1][y1]
            count += 1
        
        for j in range(swap_dest):
            x1, y1 = path[n-j][0], path[n-j][1]
            x2, y2 = path[n-j-1][0], path[n-j-1][1]
            new_grid[x1][y1], new_grid[x2][y2] = new_grid[x2][y2], new_grid[x1][y1]
            count += 1

        grids.append(new_grid)

    return grids

# Gate Level Interaction Routing

In [30]:
def rearrange_grid(grid, q1, q2, swaps):
    p1 = find_qubit(q1, grid)
    p2 = find_qubit(q2, grid)

    np = nPaths(p1, p2)
    nt = nTrace(p1, p2)
    pathList = find_min_paths(grid, p1, p2)
    # print(f"np: {np}, nt: {nt}, pathList: {pathList}")

    all_possible_grids = []
    for path in pathList:
        new_grids = swap_qubits(copy.deepcopy(grid), path)
        for new_grid in new_grids:
            all_possible_grids.append(new_grid)
    return (grid, all_possible_grids, swaps+nt)


# Main Function

In [31]:
filename = input("Enter the filename: ")
# filename = 'cnt3-5_180'
with open(f'benchmarks/revlib_decomposed/{filename}.real', 'r') as f:
    data = f.read()

qubits = int(re.search(r".numvars ([\d]*)", data).group(1))
gates = re.search(r"(?s)(?<=.begin).*?(?=.end)", data, re.MULTILINE).group(0)
lines = gates.strip().split('\n')

# print(qubits, lines)
adj_mat = [[0]*qubits for _ in range(qubits)]
counts = dict.fromkeys(range(1, qubits+1), 0)
interactions = []

for t, line in enumerate(lines, 1):
    params = line.split(' ')
    if len(params) == 2: continue    
    qubit1 = int(re.search(r"\d+", params[1]).group(0))
    qubit2 = int(re.search(r"\d+", params[2]).group(0))
    interactions.append((qubit1, qubit2))

    adj_mat[qubit1-1][qubit2-1] += 1/t
    counts[qubit1] += 1

print('AdjMat:')
for row in adj_mat: print(row)
qubit_priority = []
interaction_factor = lambda x: sum(adj_mat[x-1]) + sum(row[x-1] for row in adj_mat)

possible_src = [k for k, v in counts.items() if v == max(counts.values())]
qubit_priority.append(max(possible_src, key=interaction_factor))
counts.pop(qubit_priority[0])

while counts:
    last_qubit = qubit_priority[-1]
    max_val, max_qubit = 0, 0
    for k, v in counts.items():
        val = adj_mat[last_qubit-1][k-1] + adj_mat[k-1][last_qubit-1]
        if val > max_val:
            max_val, max_qubit = val, k
        elif val == max_val:
            if val == 0 or counts[k] > counts[max_qubit]:
                max_qubit = k

    qubit_priority.append(max_qubit)
    counts.pop(max_qubit)

print('Priority:', qubit_priority)
M, N = map(int, input("Enter the grid size: ").split())
# M, N = 3, 6
grid = place_qubits(qubit_priority, M, N, adj_mat)
print('Grid:')
for row in grid: print(row)

# rearrange_grid(grid, 1, 5, 2)
limit = int(input("Enter the limit: "))
# limit = 69

AdjMat:
[0, 0.045049663597923444, 0.059630621308120066, 0.07786696807369113, 0.044210390881327555, 0.015157542691179965, 0.016476906009483084, 0.003129959442779051, 0, 0.0009545423820068809]
[0.009233722133641434, 0, 0.11788368197097719, 0.14332002377365893, 0.09088946766415594, 0.017360313229866003, 0.019537841250596738, 0.0032000737296987323, 0, 0.00837706056899812]
[0.01531327893507839, 0.035298526099854434, 0, 0.4646855189264995, 0.30757253171400634, 0.01885386917286938, 0.02436763639988558, 0.0034188933305280876, 0, 0.04780890597175332]
[0.01873519123301177, 0.047174839987308356, 0.09476504080735376, 0, 1.2881238246787539, 0.019220857900720416, 0.033016523802970064, 0.003853693308384567, 0, 0.28434977981615983]
[0.01952613250684536, 0.06356871832424436, 0.12483336842283838, 0.03075079937642141, 0, 0.01800886928232804, 0.053017925447144225, 0.0046840719613865725, 0, 2.038641899550437]
[0.0019061294272581003, 0.021488667995899983, 0.08469519523328135, 0.2669730838986766, 0.039179929

In [32]:
leaves = [([], grid, 0)] # (parent, grids, swaps)
for i, interaction in enumerate(interactions, 1):
    print(f"Line {i}: {interaction}")
    q1, q2 = interaction
    current, excluded = [], []
    for i in leaves:
        if ManDist(find_qubit(q1, i[1]), find_qubit(q2, i[1])) > 0:
            current.append(i)
        else:
            excluded.append(i)
    new_leaves = []

    for i in current:
        parent, grid, swaps = i
        new_parent, new_grids, new_swaps = rearrange_grid(grid, q1, q2, swaps)
        new_leaves.extend([(parent+[new_parent], new_grid, new_swaps) for new_grid in new_grids if new_swaps <= limit])

    leaves = excluded + new_leaves
    tot_leaves = len(leaves)
    if tot_leaves <= 30000:
        print(f"Leaves: {len(leaves)}")
    else:
        print(f"Leaves: > 30000, pruning {tot_leaves - 30000}...")
        leaves.sort(key=lambda x: x[2])
        leaves = leaves[:30000]

Line 1: (5, 10)
Leaves: 1
Line 2: (9, 5)
Leaves: 3
Line 3: (5, 10)
Leaves: 14
Line 4: (4, 5)
Leaves: 48
Line 5: (8, 4)
Leaves: 184
Line 6: (4, 5)
Leaves: 483
Line 7: (7, 4)
Leaves: 4040
Line 8: (6, 4)
Leaves: 21814
Line 9: (6, 7)
Leaves: > 30000, pruning 85601...
Line 10: (7, 4)
Leaves: > 30000, pruning 63124...
Line 11: (4, 5)
Leaves: > 30000, pruning 101807...
Line 12: (8, 4)
Leaves: > 30000, pruning 100917...
Line 13: (4, 5)
Leaves: > 30000, pruning 34854...
Line 14: (5, 10)
Leaves: > 30000, pruning 98309...
Line 15: (9, 5)
Leaves: > 30000, pruning 140374...
Line 16: (5, 10)
Leaves: > 30000, pruning 57470...
Line 17: (4, 5)
Leaves: > 30000, pruning 69003...
Line 18: (8, 4)
Leaves: > 30000, pruning 59226...
Line 19: (4, 5)
Leaves: > 30000, pruning 22173...
Line 20: (7, 4)
Leaves: > 30000, pruning 78578...
Line 21: (6, 7)
Leaves: > 30000, pruning 86776...
Line 22: (6, 4)
Leaves: > 30000, pruning 73987...
Line 23: (7, 4)
Leaves: > 30000, pruning 52738...
Line 24: (4, 5)
Leaves: > 30000

In [33]:
# Find the minimum swaps from the current grids
min_swaps = int(1e9)
for i in leaves:
    # print(i)
    parent, grids, swaps = i
    if swaps < min_swaps:
        min_swaps = swaps
        min_grid = grids
        min_parent = parent

print(f"Minimum swaps: {min_swaps}")
print("Path: ")
for parent in min_parent:
    for row in parent: print(row)
    print()
for row in min_grid: print(row)

Minimum swaps: 872
Path: 
[0, 9, 0]
[6, 8, 7]
[3, 4, 2]
[10, 5, 1]

[0, 9, 0]
[6, 5, 7]
[3, 8, 2]
[10, 4, 1]

[0, 9, 0]
[10, 5, 7]
[6, 8, 2]
[3, 4, 1]

[0, 9, 0]
[10, 5, 7]
[6, 4, 2]
[3, 8, 1]

[0, 9, 0]
[10, 5, 2]
[6, 4, 7]
[3, 8, 1]

[0, 9, 0]
[10, 5, 2]
[4, 6, 7]
[3, 8, 1]

[0, 9, 0]
[10, 5, 2]
[6, 4, 7]
[3, 8, 1]

[0, 9, 0]
[10, 5, 2]
[6, 7, 4]
[3, 8, 1]

[0, 9, 0]
[10, 5, 2]
[6, 4, 7]
[3, 8, 1]

[0, 9, 0]
[10, 4, 2]
[6, 5, 7]
[3, 8, 1]

[0, 9, 0]
[10, 4, 2]
[6, 3, 7]
[8, 5, 1]

[0, 9, 0]
[10, 4, 2]
[8, 3, 7]
[6, 5, 1]

[0, 9, 0]
[10, 4, 2]
[8, 3, 1]
[6, 5, 7]

[0, 9, 0]
[10, 4, 2]
[8, 3, 1]
[6, 7, 5]

[0, 9, 0]
[10, 4, 2]
[8, 3, 5]
[6, 7, 1]

[0, 9, 0]
[10, 5, 2]
[8, 4, 3]
[6, 7, 1]

[0, 9, 0]
[10, 5, 2]
[8, 3, 4]
[6, 7, 1]

[0, 9, 0]
[10, 5, 2]
[8, 3, 1]
[6, 7, 4]

[0, 9, 0]
[10, 5, 2]
[8, 3, 4]
[6, 7, 1]

[0, 9, 0]
[10, 5, 2]
[8, 4, 3]
[6, 7, 1]

[0, 9, 0]
[10, 5, 2]
[8, 4, 1]
[6, 7, 3]

[0, 9, 0]
[10, 5, 2]
[8, 4, 3]
[6, 7, 1]

[0, 9, 0]
[10, 5, 3]
[8, 4, 2]
[6, 7, 1]

[0, 9, 0